In [406]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from datetime import tzinfo, timedelta, datetime
import yfinance as yf

Set_start_date = '2024-06-01'


In [407]:
#pull in data sources for trades and SP500 list
submission= pd.read_csv("/Users/maria/Documents/CodeOpDocs/Milestones/Indiv_Project/SUBMISSION.tsv", delimiter='\t')
non_deriv_trans= pd.read_csv("/Users/maria/Documents/CodeOpDocs/Milestones/Indiv_Project/NONDERIV_TRANS.tsv", delimiter='\t')
reporting_owner= pd.read_csv("/Users/maria/Documents/CodeOpDocs/Milestones/Indiv_Project/REPORTINGOWNER.tsv", delimiter='\t')

SNP= pd.read_csv("sp500_companies.csv")


C:\Users\maria\AppData\Local\Temp\ipykernel_2648\1469108220.py:3: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  non_deriv_trans= pd.read_csv("/Users/maria/Documents/CodeOpDocs/Milestones/Indiv_Project/NONDERIV_TRANS.tsv", delimiter='\t')


### make is so ACCESSION NUMBER is not Duplicated in Owner


In [408]:
#make it so there is only ONE joined owner per ACCESSION_number

reporting_owner.groupby('ACCESSION_NUMBER').agg({'RPTOWNERNAME': ','.join})
owner_agg=reporting_owner.groupby('ACCESSION_NUMBER').agg({'RPTOWNERNAME': ','.join})


In [409]:

owner_agg=owner_agg.reset_index()
owner_agg[owner_agg['ACCESSION_NUMBER']=='0001415889-24-024119']

,ACCESSION_NUMBER,RPTOWNERNAME
23583,0001415889-24-024119,"Avalon Ventures XI, L.P.,Lichter Jay,Avalon Bi..."



insider trade filing from EDGAR on https://www.sec.gov/data-research/sec-markets-data/insider-transactions-data-sets
SP500 stocks from Kaggle https://www.kaggle.com/datasets/andrewmvd/sp-500-stocks/data

#Discovering data I need


Reporting Period for American Listed Companies
n the US, insiders must file an initial filing on Form 3 within 10 days of becoming an officer, director, or beneficial owner. Changes in insider stock ownership are reported on Form 4 and must be filed with the SEC within two business days. Insiders must file a Form 5 to report any transactions that should have been reported earlier on a Form 4 or were eligible for deferred reporting. If a Form 5 must be filed, it is due 45 days after the end of the company’s fiscal year. For a more detailed explanation of the different insider reporting forms (3, 4, and 5) from the SEC, click here. 
https://undervaluedequity.com/insider-trading-rules-when-do-insiders-need-to-report-their-change-in-stock-ownership/#:~:text=Insiders%20must%20file%20a%20Form%205%20to%20report,after%20the%20end%20of%20the%20company%E2%80%99s%20fiscal%20year.


### Looking into the SP500 data to make sure enough large stocks are in my dataset

In [410]:
#Exploring SP500 data
SNP['Symbol']
universe= submission['ISSUERTRADINGSYMBOL']

In [411]:
#make list of stocks in my universe and in SP500
combined= []
for stock in universe:
    if stock in SNP['Symbol'].values:
        combined.append(stock)

print(combined)


['VLTO', 'HIG', 'MDT', 'NOW', 'L', 'MKC', 'DRI', 'DRI', 'DRI', 'PCAR', 'NOW', 'HIG', 'L', 'HIG', 'HBAN', 'AZO', 'DRI', 'RTX', 'ODFL', 'VLTO', 'AMP', 'DRI', 'CNP', 'HIG', 'HCA', 'DRI', 'HCA', 'CNP', 'LW', 'BMY', 'AZO', 'NOW', 'LW', 'PNR', 'HIG', 'L', 'AVY', 'MDT', 'PAYC', 'MDT', 'CRM', 'PAYC', 'HBAN', 'BX', 'HIG', 'LW', 'MKC', 'TEL', 'BMY', 'LW', 'MDT', 'DRI', 'LW', 'HIG', 'MDT', 'LW', 'DRI', 'MDT', 'WTW', 'FFIV', 'DRI', 'FFIV', 'TMO', 'DRI', 'MDT', 'MDT', 'NEM', 'AMGN', 'LW', 'HIG', 'DRI', 'NSC', 'NSC', 'NSC', 'NSC', 'AOS', 'TYL', 'PAYC', 'VTR', 'NVDA', 'RTX', 'RTX', 'HIG', 'NSC', 'NSC', 'NSC', 'NSC', 'CINF', 'DHI', 'ISRG', 'AOS', 'CINF', 'HLT', 'PGR', 'VZ', 'AEP', 'HLT', 'HLT', 'VZ', 'WMT', 'SYY', 'TROW', 'EXPE', 'TROW', 'TGT', 'ON', 'TROW', 'TER', 'A', 'VZ', 'TROW', 'ABBV', 'HRL', 'HUM', 'K', 'PAYC', 'HLT', 'PNC', 'KVUE', 'PANW', 'TROW', 'TROW', 'VZ', 'HLT', 'HLT', 'HLT', 'EA', 'DIS', 'VRSN', 'TER', 'HLT', 'WMT', 'VZ', 'LULU', 'TROW', 'VZ', 'DELL', 'DRI', 'GOOGL', 'MU', 'VZ', 'ADSK',

### Pulling out only the important information from each of the EDGAR files and doing a Merge

In [412]:
# merge dataFrames
combo=submission.merge(non_deriv_trans,on='ACCESSION_NUMBER', how='inner')
combo=combo.merge(owner_agg,on='ACCESSION_NUMBER', how='inner')

In [413]:
combo= combo.iloc[:][['ACCESSION_NUMBER','ISSUERTRADINGSYMBOL','TRANS_DATE','TRANS_FORM_TYPE',
                      'TRANS_CODE','TRANS_SHARES','TRANS_PRICEPERSHARE',
                      'ACCESSION_NUMBER','RPTOWNERNAME']]


In [414]:
combo.shape

(55881, 9)

In [415]:
#make a field to say whether data in SP500 in the submission data
combo['is_sp500']= combo['ISSUERTRADINGSYMBOL'].isin(combined)

#remove those not in SP500
combo=combo[combo['is_sp500']==True]
combo.shape

(11422, 10)

In [416]:
#calculate how many data points are of each transaction type
combo['TRANS_CODE'].value_counts(True)

#remove those transactions which are not S or P

combo=combo[(combo['TRANS_CODE']=='P') | (combo['TRANS_CODE']=='S')]
combo.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5669 entries, 0 to 55864
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ACCESSION_NUMBER     5669 non-null   object 
 1   ISSUERTRADINGSYMBOL  5669 non-null   object 
 2   TRANS_DATE           5669 non-null   object 
 3   TRANS_FORM_TYPE      5669 non-null   int64  
 4   TRANS_CODE           5669 non-null   object 
 5   TRANS_SHARES         5669 non-null   float64
 6   TRANS_PRICEPERSHARE  5669 non-null   float64
 7   ACCESSION_NUMBER     5669 non-null   object 
 8   RPTOWNERNAME         5669 non-null   object 
 9   is_sp500             5669 non-null   bool   
dtypes: bool(1), float64(2), int64(1), object(6)
memory usage: 448.4+ KB


In [417]:
# Putting date to datetime format
combo['TRANS_DATE']=pd.to_datetime(combo['TRANS_DATE'],format="%d-%b-%Y")
combo.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5669 entries, 0 to 55864
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ACCESSION_NUMBER     5669 non-null   object        
 1   ISSUERTRADINGSYMBOL  5669 non-null   object        
 2   TRANS_DATE           5669 non-null   datetime64[ns]
 3   TRANS_FORM_TYPE      5669 non-null   int64         
 4   TRANS_CODE           5669 non-null   object        
 5   TRANS_SHARES         5669 non-null   float64       
 6   TRANS_PRICEPERSHARE  5669 non-null   float64       
 7   ACCESSION_NUMBER     5669 non-null   object        
 8   RPTOWNERNAME         5669 non-null   object        
 9   is_sp500             5669 non-null   bool          
dtypes: bool(1), datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 448.4+ KB


In [418]:
#only focus on transactions where the transaction_form_type is 4 (which are regularly reported trades)

combo=combo[combo['TRANS_FORM_TYPE']==4]
combo.shape

(5668, 10)

In [419]:
combo.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5668 entries, 0 to 55864
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ACCESSION_NUMBER     5668 non-null   object        
 1   ISSUERTRADINGSYMBOL  5668 non-null   object        
 2   TRANS_DATE           5668 non-null   datetime64[ns]
 3   TRANS_FORM_TYPE      5668 non-null   int64         
 4   TRANS_CODE           5668 non-null   object        
 5   TRANS_SHARES         5668 non-null   float64       
 6   TRANS_PRICEPERSHARE  5668 non-null   float64       
 7   ACCESSION_NUMBER     5668 non-null   object        
 8   RPTOWNERNAME         5668 non-null   object        
 9   is_sp500             5668 non-null   bool          
dtypes: bool(1), datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 448.3+ KB


In [420]:
# Remove transactions where either the Transaction price is zero or NA

combo=combo[combo['TRANS_PRICEPERSHARE']!=0]
combo=combo.dropna(how='any',subset=['TRANS_PRICEPERSHARE'])
combo.shape

(5668, 10)

In [421]:
# Remove transactions outside the study period, in this case outside Q3 2024 as assume those are late submissions

combo=combo[(combo['TRANS_DATE']>=Set_start_date)]
combo.shape

(5629, 10)

In [422]:
# Calculating cost of transacted shares per line item (to be used later for getting avg cost of transacted stock)
combo['value_transacted']= combo['TRANS_SHARES'] * combo['TRANS_PRICEPERSHARE']


## Retrieving Stock data

In [423]:
combo.iloc[:]['TRANS_DATE'].describe()

count                             5629
mean     2024-08-11 11:04:21.573991680
min                2024-06-27 00:00:00
25%                2024-07-25 00:00:00
50%                2024-08-12 00:00:00
75%                2024-08-30 00:00:00
max                2024-09-30 00:00:00
Name: TRANS_DATE, dtype: object

In [424]:
#get the full list of stocks we need data for
#get rid of repeats using set and list
stock_selection=combo['ISSUERTRADINGSYMBOL']
stock_selection=set(stock_selection)
stock_selection=list(stock_selection) + ["ES"]


In [425]:
#pull in stock data from what was Yahoo Finance
data = yf.download(stock_selection, period="1y")



[*********************100%***********************]  372 of 372 completed


In [426]:
data.columns

MultiIndex([('Adj Close',    'A'),
            ('Adj Close', 'AAPL'),
            ('Adj Close', 'ABBV'),
            ('Adj Close', 'ABNB'),
            ('Adj Close',  'ABT'),
            ('Adj Close', 'ACGL'),
            ('Adj Close',  'ACN'),
            ('Adj Close', 'ADBE'),
            ('Adj Close',  'ADI'),
            ('Adj Close',  'ADM'),
            ...
            (   'Volume',  'WEC'),
            (   'Volume',   'WM'),
            (   'Volume',  'WMT'),
            (   'Volume',  'WST'),
            (   'Volume',   'WY'),
            (   'Volume', 'WYNN'),
            (   'Volume',  'XEL'),
            (   'Volume',  'XYL'),
            (   'Volume',  'YUM'),
            (   'Volume', 'ZBRA')],
           names=['Price', 'Ticker'], length=2232)

In [427]:
#select only close prices
data=data['Close']

In [428]:
#remove time-zone data from 'data' so it can be indexed

data=data.tz_convert(None)


## Shifting my stock prices so that I can see price development after the transaction

In [429]:
#getting stock prices in the day before and the 5 days after
data_neg_1=data.shift(periods=1)
data_1=data.shift(periods=-1)
data_2=data.shift(periods=-2)
data_3=data.shift(periods=-3)
data_4=data.shift(periods=-4)
data_5=data.shift(periods=-5)

## Aggregating Transaction Data

In [430]:
#do the same using a pivot table so that i can pull out the sum TRANS_SHARE and avg TRANS_PRICEPERSHARE

piv_table=pd.pivot_table(combo,index=['TRANS_DATE','ISSUERTRADINGSYMBOL'],aggfunc={'TRANS_SHARES':np.sum,'value_transacted':np.sum})


C:\Users\maria\AppData\Local\Temp\ipykernel_2648\1044570992.py:3: FutureWarning: The provided callable <function sum at 0x0000026F99633560> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  piv_table=pd.pivot_table(combo,index=['TRANS_DATE','ISSUERTRADINGSYMBOL'],aggfunc={'TRANS_SHARES':np.sum,'value_transacted':np.sum})
C:\Users\maria\AppData\Local\Temp\ipykernel_2648\1044570992.py:3: FutureWarning: The provided callable <function sum at 0x0000026F99633560> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  piv_table=pd.pivot_table(combo,index=['TRANS_DATE','ISSUERTRADINGSYMBOL'],aggfunc={'TRANS_SHARES':np.sum,'value_transacted':np.sum})


In [431]:
#here i am pulling out the average price transacted in a day by insiders at that company
piv_table['transacted_avg']=piv_table['value_transacted']/piv_table['TRANS_SHARES']

In [432]:

piv_table.to_csv("/Users/maria/Desktop/csv_files/piv_table.csv",index=True, sep=',')

combo.to_csv("/Users/maria/Desktop/csv_files/combo.csv",index=True, sep=',')

data.to_csv("/Users/maria/Desktop/csv_files/data.csv",index=True, sep=',')

In [433]:
#I reset the index so that we can iterate through them for my calculations 
piv_table=piv_table.reset_index()

In [434]:
#using iterrows we will now iterate through to calculate columns 
#with stock prices at time t-1, t0, t+1, t+2,t+3,t+4,t+5


#calc t-1 (day before the transaction)

for idx, row in piv_table.iterrows():
    trans_date=row['TRANS_DATE']
    symbol= row['ISSUERTRADINGSYMBOL']
    try:    
        value=data_neg_1.loc[trans_date,symbol]
        piv_table.loc[idx,"STOCK_-1"]=value
    except:
        piv_table.loc[idx,"STOCK_-1"]=np.nan

#calc t0 (day of the transaction)

for idx, row in piv_table.iterrows():
    trans_date=row['TRANS_DATE']
    symbol= row['ISSUERTRADINGSYMBOL']
    try:    
        value=data.loc[trans_date,symbol]
        piv_table.loc[idx,"STOCK_t0"]=value
    except:
        piv_table.loc[idx,"STOCK_t0"]=np.nan

#calc t+1 (day after  the transaction)
for idx, row in piv_table.iterrows():
    trans_date=row['TRANS_DATE']
    symbol= row['ISSUERTRADINGSYMBOL']
    try:    
        value=data_1.loc[trans_date,symbol]
        piv_table.loc[idx,"STOCK_t1"]=value
    except:
        piv_table.loc[idx,"STOCK_t1"]=np.nan

#calc t+2 (day 2 after  the transaction)        

for idx, row in piv_table.iterrows():
    trans_date=row['TRANS_DATE']
    symbol= row['ISSUERTRADINGSYMBOL']
    try:    
        value=data_2.loc[trans_date,symbol]
        piv_table.loc[idx,"STOCK_t2"]=value
    except:
        piv_table.loc[idx,"STOCK_t2"]=np.nan


#calc t+3 (day 3 after the transaction)   
for idx, row in piv_table.iterrows():
    trans_date=row['TRANS_DATE']
    symbol= row['ISSUERTRADINGSYMBOL']
    try:    
        value=data_3.loc[trans_date,symbol]
        piv_table.loc[idx,"STOCK_t3"]=value
    except:
        piv_table.loc[idx,"STOCK_t3"]=np.nan      


#calc t+4 (day 4 after the transaction)   
for idx, row in piv_table.iterrows():
    trans_date=row['TRANS_DATE']
    symbol= row['ISSUERTRADINGSYMBOL']
    try:    
        value=data_4.loc[trans_date,symbol]
        piv_table.loc[idx,"STOCK_t4"]=value
    except:
        piv_table.loc[idx,"STOCK_t4"]=np.nan    

#calc t+5 (day 5 after the transaction)   
for idx, row in piv_table.iterrows():
    trans_date=row['TRANS_DATE']
    symbol= row['ISSUERTRADINGSYMBOL']
    try:    
        value=data_5.loc[trans_date,symbol]
        piv_table.loc[idx,"STOCK_t5"]=value
    except:
        piv_table.loc[idx,"STOCK_t5"]=np.nan   

In [435]:
piv_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1782 entries, 0 to 1781
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   TRANS_DATE           1782 non-null   datetime64[ns]
 1   ISSUERTRADINGSYMBOL  1782 non-null   object        
 2   TRANS_SHARES         1782 non-null   float64       
 3   value_transacted     1782 non-null   float64       
 4   transacted_avg       1782 non-null   float64       
 5   STOCK_-1             1780 non-null   float64       
 6   STOCK_t0             1780 non-null   float64       
 7   STOCK_t1             1780 non-null   float64       
 8   STOCK_t2             1780 non-null   float64       
 9   STOCK_t3             1780 non-null   float64       
 10  STOCK_t4             1780 non-null   float64       
 11  STOCK_t5             1780 non-null   float64       
dtypes: datetime64[ns](1), float64(10), object(1)
memory usage: 167.2+ KB
